# Sympy en profundidad
Sympy provee de tantas herramientas que siempre hay impresion cada vez que se nombra una funcion, 

Exploraremos en este notebook:
- Symbols
- Rewriting (util para reescribir terminos en otros, idealmente braket deberia inyectarse en este medio)
- Integrals
- Hypergeometrics

In [10]:
import sympy as sp

# Symbols
- `Dummy` 
- 

`Dummy: class sympy.core.symbol.Dummy`
cada definicion es unica, por tanto no hay choques si tenemos dos dummy index llamados `i`

In [11]:
print(sp.Dummy) # shortcut

j = sp.Dummy("j")
j == sp.Dummy("j") # comparandolo a una nueva instancia

<class 'sympy.core.symbol.Dummy'>


False

`var: sympy.core.symbol.var` crea un simbolo y lo inyecta al global, de manera que puede ser accedido en cualquier lugar.

Es recomendado utilizar `symbols()` en librerias, para no importar variables inesperadas al scope global

In [20]:
x, y = sp.symbols('x y')
print(x)
# rangos
print(sp.symbols('x:10'))
print(sp.symbols('x:z'))
# multiples rangos
print(sp.symbols('x:2(1:3)'))

x
(x0, x1, x2, x3, x4, x5, x6, x7, x8, x9)
(x, y, z)
(x01, x02, x11, x12)


In [28]:
# es posible crear instancias de clases generales
f,g,h = sp.symbols('f:h', cls=sp.Function)

# tambien agregar o quitar la conmutatividad, que viene True por default
A, B = sp.symbols('A:B', conmutative=False)

# Rewriting


In [31]:
x, y, z = sp.symbols('x:z')

# expansion basica de expresiones
((x + y)*(x - y)).expand(basic=True)

x**2 - y**2

In [30]:
((x + y + z)**2).expand(basic=True) #basic viene por default, es expandir algebraicamente

x**2 + 2*x*y + 2*x*z + y**2 + 2*y*z + z**2

In [38]:
# es posible expandir en complejos
# notese que aqui no especficiamos si 'x' o 'y' son reales
# por ello es una expresion mas larga
((x + sp.I * y)).expand(complex=True)

re(x) + I*re(y) + I*im(x) - im(y)

In [39]:
x, y = sp.symbols('x y', real=True)
# al haber especificado es mas simple
((x + sp.I * y)).expand(complex=True)

x + I*y

### common sub expression detection and collection
Antes de evaluar algo muy grande, podemos identificar las sub expresiones que se repiten, de manera de solo hacer una evaluacion y luego juntar los calculos

`sp.cse()`


In [48]:
expr1 = sp.sqrt( sp.sin(x) + sp.sqrt(sp.sin(x)) ) # tengamos esta expresion
expr1

sqrt(sqrt(sin(x)) + sin(x))

In [49]:
sp.cse(expr1) # junta expresiones comunes

([(x0, sin(x))], [sqrt(sqrt(x0) + x0)])

In [51]:
sp.cse( sp.sqrt( sp.sin(x)+5)* sp.sqrt( sp.sin(x)+4))

([(x0, sin(x))], [sqrt(x0 + 4)*sqrt(x0 + 5)])

### rewritting using equations
permite reescribir condiciones en forma de ecuaciones, sin embargo no es tan poderoso como para poder aplicarse a integrales


In [62]:
from sympy import *
x, t, z = symbols('x t z')

# Define the rule for rewriting the Gamma function integral
gamma_rule = Equality( Integral(t**(x-1)*exp(-t), (t, 0, oo)), gamma(x))

# Create an expression with the Gamma function integral
expr = Integral(t**(x-1)*exp(-t), (t, 0, oo))

# Rewrite the expression using the Gamma function integral representation
rewritten_expr = expr.rewrite(gamma_rule)

# Print the result
rewritten_expr


Integral(t**(x - 1)*exp(-t), (t, 0, oo))

In [77]:
Integral(t**(3*x-1)*exp(-t), (t, 0, oo)).subs({Integral(t**(x-1)*exp(-t), (t, 0, oo)): gamma(x)})

Integral(t**(3*x - 1)*exp(-t), (t, 0, oo))

In [88]:
from sympy import *
x, t = symbols('x t')

# Define the expression with the Gamma function integral
expr = 2*Integral(t**(x-1)*exp(-t) * 4, (t, 0, oo))

# Define the rule for rewriting the Gamma function integral
gamma_rule = Integral(t**(x-1)*exp(-t), (t, 0, oo)).rewrite(gamma(x)  )

# Replace the integral expression with gamma(x)
replaced_expr = expr.replace(gamma_rule, gamma)

replaced_expr


2*Integral(4*t**(x - 1)*exp(-t), (t, 0, oo))

### expansion en series


In [66]:
sin(x).series(x, 0, 4)

x - x**3/6 + O(x**4)

In [68]:
sin(x).series(x, 0, 4).removeO()

-x**3/6 + x

### Construyendo el Braket

In [82]:
from sympy import *

class braket(Symbol):
    def __new__(cls, argument, **kwargs):
        # cls es una referencia a la clase siendo creada
        # re escrbimos para nuevos atributos
        obj = super().__new__(cls, str(argument), **kwargs)
        # el inside es un symbol, se asume generalidad, puede hasta ser un symbol
        obj.argument = argument
        # print(type(obj.argument)) # quick debugging
        return obj

    def __str__(self):
        return f"braket({self.argument})"

    
    def _latex(self, printer):
        """
        Permite incluir una representacion de printing de latex
        """
        return "\\langle " + printer._print(self.argument) + "\\rangle "

    def args(self):
        # experimental, al correr braket(..).args() se devuelve a si mismo
        return (self,)
    
    #def __mul__(self, other):
    #    pass

    def solve(self, symbol_to_solve : Symbol, debug_print: bool = False):
        """
        Takes input of a symbol included inside the braket, and returns the solution
        """
        solutions = solve(self.argument, symbol_to_solve)
        if debug_print:
            print(f'number of solutions: {len(solutions)}')

        if len(solutions) == 0:
            print('no solutions found')
        else:
            return solutions[0]

    def coefficients(self, symbol : Symbol):
        """
        Returns the coefficients of the symbol inside the braket.
        """
        return self.argument.coeff(symbol)

    def modified(self, symbol : Symbol):
        """
        Returns a new braket object with the modified argument as described.
        """
        coeff_n = self.coefficients(symbol)
        if coeff_n == 0:
            return self
        else:
            new_arg = self.argument / coeff_n
            return coeff_n * braket(new_arg)


def get_equations(expresion):
    # Extract the equations
    equations = []
    symbols_in_braket = []

    
    if isinstance(expresion, braket):
        eq = expresion.argument
        equations.append(eq) # append as equation

        free_symbols = eq.free_symbols #see symbols inside a braket
        for symbol in free_symbols: 
            if symbol not in symbols_in_braket: # add the symbol if not already added
                symbols_in_braket.append(symbol)

        return {'symbols': symbols_in_braket, 'equations': equations}


    if isinstance(expresion, Mul):
        for term in expresion.args:
            if isinstance(term, braket):
                # get the inside of a braket
                eq = term.argument
                equations.append(eq) # append as equation

                free_symbols = eq.free_symbols #see symbols inside a braket
                for symbol in free_symbols: 
                    if symbol not in symbols_in_braket: # add the symbol if not already added
                        symbols_in_braket.append(symbol)
        return {'symbols': symbols_in_braket, 'equations': equations}




def normalize_braket(expresion):
    """
    Not neccesary, the use of going throught every braket and using braket.modified()
    but all of this occurs in the use of matrix
    """
    pass



def braket_solver(expresion):
    """
    Extracts all equations from the argument of the multibraket.
    Returns a list of equations.
    """
    
    get_equations(expresion)

    return {'symbols': symbols_in_braket, 'equations': equations, 'solutions': solutions}

In [83]:
n,i = symbols('n i')

multibraket = braket(2*n + 1) * braket(n + i) * gamma(n)
multibraket.args

(2*n + 1, i + n, gamma(n))

In [17]:
n = Symbol('n')
b2n1 = braket(2*n + 1)
b2n1.solve(n)

b2n1

b2n1.modified(n)

2*n + 1/2

### Como se resuelve varios brakets?
- usar algebra lineal de maner que se tenga
$$
\sum_{i,j,\dots} \phi_{i,j,\dots} \langle i \rangle \langle j \rangle\dots =\frac{1}{|J|} \Gamma(-n_i) \Gamma(-n_j) \dots |_{ J = \vec 0 }
$$
por hacer en esto:
- [ ] construir el extractor de ecuaciones
- [ ] reemplazador de brakets utilizando el braket solver, (suma, phi, braket) -> gamma
